In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")
df_submission = pd.read_csv("/kaggle/input/playground-series-s5e4/sample_submission.csv")

In [4]:
df_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


In [5]:
df_train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [6]:
# Extract episode number from Episode_Title
df_train['Episode_Number'] = df_train['Episode_Title'].str.extract(r'(\d+)').astype(int)

# Perform one-hot encoding on Podcast_Name
podcast_dummies = pd.get_dummies(df_train['Podcast_Name'], prefix='Podcast_Name')

# Multiply the one-hot encoded columns by the Episode_Number
podcast_dummies = podcast_dummies.mul(df_train['Episode_Number'], axis=0)

# Add the new columns to the original dataframe
df_train = pd.concat([df_train, podcast_dummies], axis=1)

# Display the updated dataframe
df_train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,...,Podcast_Name_Sports Central,Podcast_Name_Sports Weekly,Podcast_Name_Study Sessions,Podcast_Name_Style Guide,Podcast_Name_Tech Talks,Podcast_Name_Tech Trends,Podcast_Name_True Crime Stories,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,...,0,0,0,0,0,0,0,0,0,0
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,...,0,0,16,0,0,0,0,0,0,0
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,...,0,0,0,0,0,0,0,0,0,0
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Extract episode number from Episode_Title
df_test['Episode_Number'] = df_test['Episode_Title'].str.extract(r'(\d+)').astype(int)

# Perform one-hot encoding on Podcast_Name
podcast_dummies = pd.get_dummies(df_test['Podcast_Name'], prefix='Podcast_Name')

# Multiply the one-hot encoded columns by the Episode_Number
podcast_dummies = podcast_dummies.mul(df_test['Episode_Number'], axis=0)

# Add the new columns to the original dataframe
df_test = pd.concat([df_test, podcast_dummies], axis=1)

# Display the updated dataframe
df_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,...,Podcast_Name_Sports Central,Podcast_Name_Sports Weekly,Podcast_Name_Study Sessions,Podcast_Name_Style Guide,Podcast_Name_Tech Talks,Podcast_Name_Tech Trends,Podcast_Name_True Crime Stories,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,...,0,0,0,0,0,0,0,0,0,0
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,...,0,0,0,0,0,0,0,0,0,0
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,...,0,0,0,0,0,0,0,0,0,0
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
for element in df_train.columns.tolist():
    if element not in df_test.columns.tolist():
        print(element)

Listening_Time_minutes


In [9]:
df_train.isnull().sum()

id                                     0
Podcast_Name                           0
Episode_Title                          0
Episode_Length_minutes             87093
Genre                                  0
                                   ...  
Podcast_Name_Tech Trends               0
Podcast_Name_True Crime Stories        0
Podcast_Name_Tune Time                 0
Podcast_Name_Wellness Wave             0
Podcast_Name_World Watch               0
Length: 61, dtype: int64

In [10]:
df_test = df_test.set_index('id', inplace=False)
df_train = df_train.set_index('id', inplace=False)

In [11]:
X = df_train.drop(columns = ['Listening_Time_minutes', 'Podcast_Name', 'Episode_Title'], axis = 1)
y = df_train["Listening_Time_minutes"]
df_test = df_test.drop(columns = ['Podcast_Name', 'Episode_Title'], axis =1)

In [12]:
df_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Number,Podcast_Name_Athlete's Arena,...,Podcast_Name_Sports Central,Podcast_Name_Sports Weekly,Podcast_Name_Study Sessions,Podcast_Name_Style Guide,Podcast_Name_Tech Talks,Podcast_Name_Tech Trends,Podcast_Name_True Crime Stories,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch
id,,,,,,,,,,,,,,,,,,,,,
750000,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral,73,0,...,0,0,0,0,0,0,0,0,0,0
750001,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral,23,0,...,0,0,0,0,0,0,0,0,0,0
750002,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive,11,0,...,0,0,0,0,0,0,0,0,0,0
750003,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive,73,0,...,0,0,0,0,0,0,0,0,0,0
750004,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral,50,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
#categorical_columns = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
categorical_columns = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
for col in categorical_columns:
    X[col] = label_encoder.fit_transform(X[col])
    
    df_test[col] = label_encoder.transform(df_test[col])

In [14]:
df_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Number,Podcast_Name_Athlete's Arena,...,Podcast_Name_Sports Central,Podcast_Name_Sports Weekly,Podcast_Name_Study Sessions,Podcast_Name_Style Guide,Podcast_Name_Tech Talks,Podcast_Name_Tech Trends,Podcast_Name_True Crime Stories,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch
id,,,,,,,,,,,,,,,,,,,,,
750000,78.96,2,38.11,2,1,53.33,1.0,1,73,0,...,0,0,0,0,0,0,0,0,0,0
750001,27.87,5,71.29,3,2,NaN,0.0,1,23,0,...,0,0,0,0,0,0,0,0,0,0
750002,69.10,1,67.89,0,1,97.51,0.0,2,11,0,...,0,0,0,0,0,0,0,0,0,0
750003,115.39,1,23.40,3,2,51.75,2.0,2,73,0,...,0,0,0,0,0,0,0,0,0,0
750004,72.32,4,58.10,6,2,11.30,2.0,1,50,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.2, random_state=42)

In [16]:
X_train.shape

(600000, 57)

## Model XGBoost

In [17]:
params = {'lambda': 0.04186421414163593,
 'alpha': 0.010101632821320981,
 'subsample': 0.9737046239306681,
 'colsample_bytree': 0.9367557190770411,
 'max_depth': 10,
 'eta': 0.012528084926241448,
 'gamma': 7.340223431497053,
 'scale_pos_weight': 2.237856577831878,
 'min_child_weight': 3,
 'max_delta_step': 9,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'verbosity': 0,
 'booster': 'gbtree',
 'device' : 'cuda',
 'early_stopping_rounds':100,
 'n_estimators' : 18000}

In [18]:
import xgboost as xgb
from xgboost import XGBRegressor

model_xgb = XGBRegressor(**params)
model_xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

[0]	validation_0-rmse:27.06170	validation_1-rmse:27.04686
[100]	validation_0-rmse:20.23873	validation_1-rmse:20.26572
[200]	validation_0-rmse:15.71812	validation_1-rmse:15.80399
[300]	validation_0-rmse:13.50083	validation_1-rmse:13.66757
[400]	validation_0-rmse:12.86742	validation_1-rmse:13.11611
[500]	validation_0-rmse:12.69298	validation_1-rmse:13.01239
[600]	validation_0-rmse:12.61021	validation_1-rmse:12.98545
[700]	validation_0-rmse:12.54424	validation_1-rmse:12.96969
[800]	validation_0-rmse:12.48875	validation_1-rmse:12.96069
[900]	validation_0-rmse:12.43993	validation_1-rmse:12.95298
[1000]	validation_0-rmse:12.39557	validation_1-rmse:12.94694
[1100]	validation_0-rmse:12.35085	validation_1-rmse:12.94053
[1200]	validation_0-rmse:12.31050	validation_1-rmse:12.93552
[1300]	validation_0-rmse:12.27411	validation_1-rmse:12.93144
[1400]	validation_0-rmse:12.23814	validation_1-rmse:12.92767
[1500]	validation_0-rmse:12.20132	validation_1-rmse:12.92352
[1600]	validation_0-rmse:12.16717	va

XGBRegressor(alpha=0.010101632821320981, base_score=None, booster='gbtree',
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9367557190770411, device='cuda',
             early_stopping_rounds=100, enable_categorical=False,
             eta=0.012528084926241448, eval_metric='rmse', feature_types=None,
             gamma=7.340223431497053, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.04186421414163593,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=9, max_depth=10,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=18000, ...)

In [19]:
#df_test.to_device("cuda")
df_submission['Listening_Time_minutes'] = model_xgb.predict(df_test)

In [20]:
df_submission.head()

,id,Listening_Time_minutes
0,750000,52.510712
1,750001,18.414955
2,750002,48.752575
3,750003,75.687241
4,750004,46.501568


In [21]:
df_submission.to_csv('submission12.95.csv', index=False)


## Optuna

In [22]:
# import optuna
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# def objective(trial):
#     param = {
#         'objective': 'reg:squarederror',
#         'eval_metric': 'rmse',
#         'verbosity': 0,
#         'booster': 'gbtree',
#         'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
#         'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
#         'subsample': trial.suggest_float('subsample', 0.4, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'eta': trial.suggest_float('eta', 0.01, 0.3),
#         'gamma': trial.suggest_float('gamma', 0, 10),
#         'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
#         'device' : 'cuda',
#         'early_stopping_rounds':100,
#         'n_estimators' : 18000
        
#     }

#     model = xgb.XGBRegressor(**param)

#     model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)

#     # Make predictions on the test set
#     y_pred = model.predict(X_test)

#     # Calculate the root mean squared error
#     rmse = mean_squared_error(y_test, y_pred, squared=False)

#     return rmse


# # Create an Optuna study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=10)

# # Print the best hyperparameters and the corresponding RMSE
# print("Best hyperparameters:", study.best_params)
# print("Best RMSE:", study.best_value)

In [23]:
params

{'lambda': 0.04186421414163593,
 'alpha': 0.010101632821320981,
 'subsample': 0.9737046239306681,
 'colsample_bytree': 0.9367557190770411,
 'max_depth': 10,
 'eta': 0.012528084926241448,
 'gamma': 7.340223431497053,
 'scale_pos_weight': 2.237856577831878,
 'min_child_weight': 3,
 'max_delta_step': 9,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'verbosity': 0,
 'booster': 'gbtree',
 'device': 'cuda',
 'early_stopping_rounds': 100,
 'n_estimators': 18000}

In [24]:
# Best_hyperparameters.append({'objective': 'reg:squarederror',
#         'eval_metric': 'rmse',
#         'verbosity': 0,
#         'booster': 'gbtree',})

In [25]:
# import xgboost as xgb
# # from xgboost import XGBRegressor

# model_xgb = XGBRegressor(**params)
# model_xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

In [26]:


# def objective(trial):
#     """Objective function for Optuna optimization."""

#     # Define the hyperparameter search space
#     xgb_learning_rate = trial.suggest_float("xgb_learning_rate", 0.01, 0.1)
#     xgb_max_depth = trial.suggest_int("xgb_max_depth", 3, 10)
#     xgb_n_estimators = trial.suggest_int("xgb_n_estimators", 1000, 20000, step=500)
#     #xgb_subsample = trial.suggest_float("xgb_subsample", 0.5, 1.0)
#     #xgb_colsample_bytree = trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0)

#     # Define the XGBoost regressor with the suggested hyperparameters
#     model_xgb = xgb.XGBRegressor(
#         objective='reg:squarederror',
#         n_estimators=xgb_n_estimators,
#         learning_rate=xgb_learning_rate,
#         max_depth=xgb_max_depth,
#         random_state=42,
#         tree_method='hist',  # Use hist for faster training
#         device = "cuda",
#         #subsample=xgb_subsample,
#         #colsample_bytree=xgb_colsample_bytree,
#         early_stopping_rounds=100
#     )

#     # Train the model and evaluate on the validation set
#     model_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

#     # Make predictions on the test set
#     y_pred = model_xgb.predict(X_test)

#     # Calculate the root mean squared error
#     rmse = mean_squared_error(y_test, y_pred, squared=False)

#     return rmse


# # Create an Optuna study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=10)

# # Print the best hyperparameters and the corresponding RMSE
# print("Best hyperparameters:", study.best_params)
# print("Best RMSE:", study.best_value)

# # Train the final model with the best hyperparameters
# best_model_xgb = xgb.XGBRegressor(
#     objective='reg:squarederror',
#     n_estimators=study.best_params['xgb_n_estimators'],
#     learning_rate=study.best_params['xgb_learning_rate'],
#     max_depth=study.best_params['xgb_max_depth'],
#     random_state=42,
#     tree_method='hist',  # Use hist for faster training
#     device = "cuda",
#     #subsample=study.best_params['xgb_subsample'],
#     #colsample_bytree=study.best_params['xgb_colsample_bytree'],
#     early_stopping_rounds=None  # Disable early stopping for final training
# )

# best_model_xgb.fit(X_train, y_train)